# Model IO模块课程演练

In [1]:
## 导入依赖库

!pip install langchain

## 1. 什么是LCEL?

LangChain于8月1日0.254版本更新，声称采用新的语法来创建带和组合功能的Chain，同时提供一个新的接口，支持批处理、异步和流处理，将这种语法成
为LangChain Expression Language(LCEL)

## 2. Model

### 2.1 Model的分类

LLMs: LangChain 的核心组件。LangChain并不提供自己的LLMs，而是为与许多不同的LLMs（OpenAI、Cohere、Hugging Face等）进行交互提供了一个标准
接口。（类似于Completion）

Chat Models: 语言模型的一种变体。虽然聊天模型在内部使用了语言模型，但它们提供的接口略有不同。与其暴露一个“输入文本，输出文本”的API不同，
它们提供了一个以“聊天消息”作为输入和输出的接口。(类似于Chat Completion)

### 2.2 LLMs

In [1]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo",openai_api_base = "https://newone.nxykj.tech/v1")

llm.invoke("什么是机器学习？")

D:\soft\python\Lib\site-packages\langchain\llms\openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
D:\soft\python\Lib\site-packages\langchain\llms\openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'机器学习是一种人工智能（AI）的分支领域，旨在让机器通过从数据中学习来改善性能，而不是显式地编程。机器学习涉及开发算法和模型，使机器能够通过数据获取知识和经验，并根据这些知识和经验来做出预测或做出决策。机器学习方法基于统计学、优化算法和计算机科学的原理。它可以应用于各个领域，例如图像和语音识别、自然语言处理、推荐系统、金融预测等。机器学习可以分为监督学习、无监督学习和强化学习等不同类型。'

In [2]:
llm("什么是大模型")

'大模型是指指数级别或更大的模型，通常需要大量的计算资源和存储空间来训练和部署。大模型通常包含数千万或数亿个参数，能够处理复杂的任务和数据，如自然语言处理，语音识别，计算机视觉等领域。大模型的典型例子包括深度学习模型，如BERT、GPT-3等。大模型的训练和使用需要高性能计算技术和高效的算法设计，是人工智能和机器学习领域的热门研究方向之一。'

OpenAI API 地址：https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html#langchain.llms.openai.OpenAI

### 2.3 Chat Model

In [4]:


from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base = "https://newone.nxykj.tech/v1")

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

messages = [SystemMessage(content="你是一个智能助手"),
 HumanMessage(content="第二十一届世界杯在哪儿举行的?"),
 AIMessage(content="在俄罗斯"), 
 HumanMessage(content="冠军是哪个球队")]

In [5]:
chat_model.invoke(messages)

AIMessage(content='法国队是第二十一届世界杯的冠军。')

In [5]:
chat_model.invoke(messages).content

'法国获得了第二十一届世界杯的冠军。'

In [6]:
chat_model(messages)

AIMessage(content='2018年世界杯的冠军是法国队。')

ChatOpenAI API地址：https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html#langchain.chat_models.openai.ChatOpenAI

## 3.prompt

一个语言模型的提示是用户提供的一组指令或输入，用于引导模型的响应，帮助它理解上下文并生成相关和连贯的基于语言的输出，例如回答问题、完成句子或
进行对话。

提示模板（Prompt Templates）：参数化的模型输入

示例选择器（Example Selectors）：动态选择要包含在提示中的示例

### 3.1 Prompt Templates

#### 1. use PromptTemplate

In [6]:
## 可以动态传入参数
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [8]:
## 也可以不传参数
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

In [9]:
llm(prompt_template.format())

'Why did the tomato turn red?\n\nBecause it saw the salad dressing!'

#### 2.use ChatPromptTemplate

In [10]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")

chat_model.invoke(messages)

AIMessage(content='My name is Bob. How can I assist you today?')

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "你是一个智能助手，让用户变得更积极"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

chat_model(chat_template.format_messages(text="我最近比较烦"))

AIMessage(content='我了解，人们常常会经历烦恼和困扰。但是，积极的态度和心态可以帮助我们应对困难和压力。让我来分享一些帮助你变得更积极的建议：\n\n1. 寻找支持：与朋友、家人或专业人士交流，分享你的烦恼，听取他们的建议和支持。\n\n2. 培养感恩之心：每天花一些时间思考你所拥有的，珍惜和感激这些东西。感恩之心可以帮助你更加积极地看待生活。\n\n3. 设定目标：制定一些具体的、可量化的目标，并制定实现这些目标的计划。追求目标可以给你带来成就感和动力。\n\n4. 培养健康的生活习惯：保持良好的饮食、充足的睡眠和适度的运动可以提升你的身体和心理健康。\n\n5. 积极思考：尝试改变消极的思维习惯，转而寻找积极的解释和观点。积极思考可以帮助你更好地应对挑战和困难。\n\n6. 寻找乐趣和爱好：找到一些你真正喜欢的活动或爱好，并定期参与其中。这样可以让你感到充实和快乐。\n\n7. 培养自我关爱：给自己一些时间和空间，关注自己的需求和感受。学会放松和照顾自己，这样你将更有能力面对困难。\n\n请记住，积极的态度需要时间和努力来培养。每天都尝试一些小的积极行为，逐渐改变你的思维方式。相信自己，你一定能够变得更积极和乐观！')

#### 3.Few-shot prompt templates

In [13]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples  = [
{
"question": "穆罕默德·阿里（Muhammad Ali）和艾伦·图灵（Alan Turing）中哪个活得更长？",
"answer":
"""
这里需要后续问题吗：是。
后续问题：穆罕默德·阿里去世时多少岁？
中间答案：穆罕默德·阿里去世时74岁。
后续问题：艾伦·图灵去世时多少岁？
中间答案：艾伦·图灵去世时41岁。
因此最终答案是：穆罕默德·阿里
"""
},
{
"question": "craigslist的创始人是何时出生的？",
"answer":
"""
这里需要后续问题吗：是。
后续问题：craigslist的创始人是谁？
中间答案：craigslist的创始人是Craig Newmark。
后续问题：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark于1952年12月6日出生。
因此最终答案是：1952年12月6日
"""
},
{
"question": "乔治·华盛顿（George Washington）的母亲的母亲父亲是谁？",
"answer":
"""
这里需要后续问题吗：是。
后续问题：乔治·华盛顿的母亲是谁？
中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿（Mary Ball Washington）。
后续问题：玛丽·鲍尔·华盛顿的父亲是谁？
中间答案：玛丽·鲍尔·华盛顿的父亲是约瑟夫·鲍尔（Joseph Ball）。
因此最终答案是：约瑟夫·鲍尔
"""
},
{
"question": "《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？",
"answer":
"""
这里需要后续问题吗：是。
后续问题：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是史蒂文·斯皮尔伯格（Steven Spielberg）。
后续问题：史蒂文·斯皮尔伯格来自哪里？
中间答案：美国。
后续问题：《皇家赌场》的导演是谁？
中间答案：《皇家赌场》的导演是马丁·坎贝尔（Martin Campbell）。
后续问题：马丁·坎贝尔来自哪里？
中间答案：新西兰。
因此最终答案是：不是
"""
}
]

In [14]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], 
    template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: 穆罕默德·阿里（Muhammad Ali）和艾伦·图灵（Alan Turing）中哪个活得更长？

这里需要后续问题吗：是。
后续问题：穆罕默德·阿里去世时多少岁？
中间答案：穆罕默德·阿里去世时74岁。
后续问题：艾伦·图灵去世时多少岁？
中间答案：艾伦·图灵去世时41岁。
因此最终答案是：穆罕默德·阿里



In [15]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="爱因斯坦的妻子比爱因斯坦大几岁？"))

Question: 穆罕默德·阿里（Muhammad Ali）和艾伦·图灵（Alan Turing）中哪个活得更长？

这里需要后续问题吗：是。
后续问题：穆罕默德·阿里去世时多少岁？
中间答案：穆罕默德·阿里去世时74岁。
后续问题：艾伦·图灵去世时多少岁？
中间答案：艾伦·图灵去世时41岁。
因此最终答案是：穆罕默德·阿里


Question: craigslist的创始人是何时出生的？

这里需要后续问题吗：是。
后续问题：craigslist的创始人是谁？
中间答案：craigslist的创始人是Craig Newmark。
后续问题：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark于1952年12月6日出生。
因此最终答案是：1952年12月6日


Question: 乔治·华盛顿（George Washington）的母亲的母亲父亲是谁？

这里需要后续问题吗：是。
后续问题：乔治·华盛顿的母亲是谁？
中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿（Mary Ball Washington）。
后续问题：玛丽·鲍尔·华盛顿的父亲是谁？
中间答案：玛丽·鲍尔·华盛顿的父亲是约瑟夫·鲍尔（Joseph Ball）。
因此最终答案是：约瑟夫·鲍尔


Question: 《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？

这里需要后续问题吗：是。
后续问题：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是史蒂文·斯皮尔伯格（Steven Spielberg）。
后续问题：史蒂文·斯皮尔伯格来自哪里？
中间答案：美国。
后续问题：《皇家赌场》的导演是谁？
中间答案：《皇家赌场》的导演是马丁·坎贝尔（Martin Campbell）。
后续问题：马丁·坎贝尔来自哪里？
中间答案：新西兰。
因此最终答案是：不是


Question: 爱因斯坦的妻子比爱因斯坦大几岁？


In [16]:
llm(prompt.format(input="爱因斯坦的妻子比爱因斯坦大几岁？"))

'这里需要后续问题吗：是。\n后续问题：爱因斯坦的妻子是谁？\n中间答案：爱因斯坦的妻子是米莉娅·玛里琳娜（Mileva Marić）。\n后续问题：米莉娅·玛里琳娜比爱因斯坦大几岁？\n中间答案：米莉娅·玛里琳娜比爱因斯坦大一岁。\n因此最终答案是：一岁。'

### 3.2 Example selectors

#### 1.use Select by length

In [18]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25,
    # The function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [19]:
## 输入较短，所以给的示例多
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


In [20]:
# 输入的内容长，給的示例少
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


## 4.Output parsers

语言模型输出文本。但是很多时候，你可能希望获得比纯文本更结构化的信息。这就是输出解析器的用处。

输出解析器是帮助结构化语言模型响应的类。一个输出解析器必须实现两个主要方法：

"获取格式指令"：返回一个包含语言模型输出应如何格式化的字符串的方法。
"解析"：接受一个字符串（假设是语言模型的响应）并将其解析为某种结构的方法。
然后还有一个可选的方法：

"带提示解析"：接受一个字符串（假设是语言模型的响应）和一个提示（假设是生成此响应的提示），并将其解析为某种结构。提示主要是在OutputParser
希望以某种方式重试或修复输出时提供的，它需要来自提示的信息来执行这些操作。

### 1.use List parser

当您想返回一个以逗号分隔的项目列表时，可以使用此输出解析器。

In [52]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI



prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

model = OpenAI(model_name="gpt-3.5-turbo",openai_api_base = "https://newone.nxykj.tech/v1")

parser = CommaSeparatedListOutputParser()
format_instructions = parser.get_format_instructions()


## LCEL
chain = prompt | model | parser
chain.invoke({"subject": "ice cream flavors"})

['Chocolate',
 'Vanilla',
 'Strawberry',
 'Cookies and Cream',
 'Mint Chocolate Chip']